# SPECFEM2D coupling with external source injection
### November 11, 2019 
### Kurama Okubo (https://github.com/kura-okubo)

## Motivation of the coupling

**Compute intermediate- or far-field radiation with high-resolved earthquake sources.**

- Complex fault geometry
- Multiple ruptures
- Coseismic off-fault damage

## Schematic of coupling

<img src="./fig/schematic_of_coupling.png" alt="schematic" width="600"/>

# Installation of SPECFEM2D

Please follow the [original document](https://github.com/geodynamics/specfem2d/blob/master/doc/USER_MANUAL/manual_SPECFEM2D.pdf).

You can find the forked version of SPECFEM2D in (https://github.com/kura-okubo/specfem2d), which has additional subroutines and parameters for the coupling.

Installation example:
```console
git clone https://github.com/kura-okubo/specfem2d.git
cd specfem2d
./configure FC=gfortran CC=gcc
make
```

Note: the original git log is removed in this forked version.

## Key subroutines & Parameters

We developped new subroutines in `src/meshfem2D` and `src/specfem2D` and added parameters associated with coupling.
Here we listed the parameters specified in `Par_fil` as below:

|variable| type |explanation |
|---|:---:|:---|
| COUPLING_IN$^{1*}$ | bool  |Output coupling elements and read external source if true |
| extori_x | float  |x origin of coupling source region |
| extori_z | float  |z origin of coupling source region |
| R_ext | float  |Radius of coupling source elements|
| dR_ext $^{2*}$ | float  |Threshold of diference in distance (R_ext-norm(cx, cz)) to detect source element|

$^{1*}$ Please specify all parameters in the list above even though `COUPLING_IN = .false.` </br>

$^{2*}$ We need to play around this parameter during meshing process to make a nice set of coupling elements which forms closed surface. Usually dR_ext = gridsize works.

#### Technical notes
Developped source files: 
- `src/meshfem2D/determine_external_source_elements.f90`
- `src/specfem2D/compute_ext_source.f90`

We also modified/added parameters in `src/`. 

# Example 1. Validation of P-SV full-space coupling

We first demonstrate how the coupling works through the example. We try to validate the coupling with comparing the far-field ground acceleration obtained by coupling to the point source model (i.e. canonical forward modeling) built in Specfem2D.   

The project directory is located at `EXAMPLES/Validation/FullSpace`. The workflow is:

1. Turn on `COUPLING_IN = .true.` in `DATA/Par_file1` and Run `xmeshfem` by `sh run_xmeshfem2D.sh` at the `EXAMPLES/Validation/FullSpace` directory.
2. Check `OUTPUT_FILES_grid/gridfile.ps` if the coupling elements (colored by red) form a closed surface. If not, play around `dR_ext` in `Par_file`. You can find the element id (used to get `iglob`), time step and location of coupling elements in  `OUTPUT_FILES_grid/externalsource.txt`, **which is used later to make external source files.**

<img src="./fig/gridfile.png" alt="schematic" width="800"/>
Figure: gridfile.ps shows the coupling elements. Check if they form a closed surface as shown in the figure above. 


3. Set receivers in `DATA/STATIONS`. To compute the external sources at coupling elements for validation, we put receivers on the coupling elements and in far-field region. Please run `make_extsource/make_stationfile_at_extsource.py` (it requires [pandas](https://pandas.pydata.org)).

4. Run point source model as true model for validation.
    - specify source type (e.g. moment tensor) and **set `factor` to be non-zero (e.g. 1.0d10) ** in `DATA/SOURCE`.
    - set `COUPLING_IN = .false.` and run by `sh run_validation_pointsource.sh`.


5. **Make External source file**

This process is important for coupling. We need to prepare the external timeseries of acceleration with correctly named files. Time step in the external timeseries has to be same with `DT` in `Par_file` (i.e. same with specfem2D forward calculation). In this validation, we create the external source files from the result of point source model at process 4 above. Run `make_extsource/make_externalsourcefile.py`

File format is following:
    - filename is `FullSpace/extsource/EXT********.dat` (XXXXXXXX is the element id in `OUTPUT_FILES_grid/externalsource.txt` with zero padding). 
    - it contains `time(s), ax(m/s/s), az (m/s/s)` for P-SV mode, or `time(s), ay(m/s/s), dummy number` for SH mode.

Note: file name and directory are fixed in the source code, so please follow the format above.

**Practically, this part is done using external softwares (e.g. HOSSedu) to compute dynamic earthquake rupture and near-field radiation.**

6. Run coupling source.
    - turn off built-in source by `factor = 0.0 #1.0d10` in `DATA/SOURCE`.
    - set `COUPLING_IN = .true.`
    **! Note ! Don't change the element size and other coupling parameters (e.g. R_ext) otherwise the element id is not synchronized at the coupling elements.**

run it by `sh run_validation_extcouple.sh`

## Result

### Source time function
<img src="./fig/STF_fullspace.png" alt="STF_fullspace" width="400"/>


### Wave field
<img src="./fig/result_fullspace_snapshots.png" alt="STF_fullspace" width="800"/>
Color contour shows the vertical velocity, Vz. As shown in the last snapshot of coupling model, the injected wave is trapped within the source region. However, it does not affect the far-field radiation as long as the coupling elements form a closed surface.


### Far-field accelerations
run `plot_result/plot_waveform_comparison.py`.
<img src="./fig/validation_full.png" alt="validation_full" width="800"/>
Figure: Comparison of far-field accelerations recoded at the stations indicated in the figure above. To remove numerical high-frequency oscillations, low-pass filter is applied on both cases. It has a good agreement between point source model and coupling model, showing the coupling with external sources works well.

# Example 2. Validation of P-SV half-space coupling

We next validate the coupling with half-space model. 
The project directory is located at `EXAMPLES/Validation/HalfSpace`. We follow the same workflow as Example 1, and here we show the results of validation.

Source time function is Gaussian shape as Example 1.

#### computational note
    - Element size: 250 (m)
    - Total number of spectral elements: 80000
    - time step: 0.007 (s)
    - Number of step: 7000
    - number of coupling element: 510
    - Computational time : ~ 15 min (single processor)

### Wave field

<img src="./fig/result_halfspace_snapshots.png" alt="STF_fullspace" width="900"/>

Figrue: snapshots of vertical velocity, Vz. Top boundary condition is free-surface, while the bottom and sides are PML absorbing boundaries.


### Far-field accelerations
<img src="./fig/validation_half.png" alt="validation_half" width="800"/>



# Conclusion
**SPECFEM2D coupling is ready to couple with external softwares.**

We may need grid convergence analysis as there is a small error in amplitude and phase associated with coupling model. 

# Acknowledgments
We acknowledge the SPECFEM2D package (https://geodynamics.org/cig/software/specfem2d/) available at [Computational Infrastructure for Geodynamics (CIG)](https://geodynamics.org/cig/).
## References


- Tromp, J., Komatitsch, D., and Liu, Q. (2008), Spectral-element and adjoint methods in seismology, Communications in Computational Physics, 3, 1, 1-32. 

- Xie, Z., Komatitsch, D., Martin, R., and Matzen, R. (2014), Improved forward wave propagation and adjoint-based sensitivity kernel calculations using a numerically stable finite-element PML, Geophys. J. Int., 198, 3, 1714-1747, doi:10.1093/gji/ggu219. 
